https://openai.com/sitemap.xml

#**1: Install All the Required Packages**

In [ ]:
# !pip -q install langchain
# !pip -q install bitsandbytes accelerate transformers
# !pip -q install datasets loralib sentencepiece
# !pip -q install pypdf
# !pip -q install sentence_transformers

In [ ]:
# !pip -q install unstructured

In [ ]:
# !pip install tokenizers

In [ ]:
# !pip install xformers

In [ ]:
# !pip install pinecone-client

#**2: Import All the Required Libraries**

In [1]:
import torch
print(torch.cuda.is_available())        #
print(torch.cuda.get_device_name(0))    # 

True
NVIDIA GeForce RTX 3060


In [2]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Pinecone
# import pinecone
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
from langchain import HuggingFacePipeline
from huggingface_hub import notebook_login
import textwrap
import sys
import os
import torch

c:\d\generative AI\LLaMA-Reader\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from huggingface_hub import login
import os
from dotenv import load_dotenv

# Load token từ .env
load_dotenv()
token = os.environ["HUGGINGFACEHUB_API_TOKEN"]

# Login trực tiếp bằng code
login(token=token)

print("✅ Đã login thành công!")

✅ Đã login thành công!


In [5]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

#**3: Pass the URLs and extract the data from these URLs**

In [4]:
URLs=[
    'https://www.mosaicml.com/blog/mpt-7b',
    'https://stability.ai/blog/stability-ai-launches-the-first-of-its-stablelm-suite-of-language-models',
    'https://lmsys.org/blog/2023-03-30-vicuna/'

]

In [5]:
loaders = UnstructuredURLLoader(urls=URLs)
data = loaders.load()

In [6]:
data

[Document(metadata={'source': 'https://www.mosaicml.com/blog/mpt-7b'}, page_content='Introducing MPT-7B, the first entry in our MosaicML Foundation Series. MPT-7B is a transformer trained from scratch on 1T tokens of text and code. It is open source, available for commercial use, and matches the quality of LLaMA-7B. MPT-7B was trained on the MosaicML platform in 9.5 days with zero human intervention at a cost of ~$200k.\n\nLarge language models (LLMs) are changing the world, but for those outside well-resourced industry labs, it can be extremely difficult to train and deploy these models. This has led to a flurry of activity centered on open-source LLMs, such as the LLaMA series from Meta, the Pythia series from EleutherAI, the StableLM series from StabilityAI, and the OpenLLaMA model from Berkeley AI Research.\n\nToday, we at MosaicML are releasing a new model series called MPT (MosaicML Pretrained Transformer) to address the limitations of the above models and finally provide a comme

In [7]:
len(data)

3

#**4: Split the Text into Chunks**

In [6]:
text_splitter=CharacterTextSplitter(separator='\n',
                                    chunk_size=1000,
                                    chunk_overlap=200)

In [7]:
text_chunks=text_splitter.split_documents(data)

In [15]:
len(text_chunks)

57

In [ ]:
text_chunks

In [20]:
text_chunks[0]

Document(metadata={'source': 'https://www.mosaicml.com/blog/mpt-7b'}, page_content='Introducing MPT-7B, the first entry in our MosaicML Foundation Series. MPT-7B is a transformer trained from scratch on 1T tokens of text and code. It is open source, available for commercial use, and matches the quality of LLaMA-7B. MPT-7B was trained on the MosaicML platform in 9.5 days with zero human intervention at a cost of ~$200k.\nLarge language models (LLMs) are changing the world, but for those outside well-resourced industry labs, it can be extremely difficult to train and deploy these models. This has led to a flurry of activity centered on open-source LLMs, such as the LLaMA series from Meta, the Pythia series from EleutherAI, the StableLM series from StabilityAI, and the OpenLLaMA model from Berkeley AI Research.')

#**5: Download the Hugging Face Embeddings**

In [8]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
# embeddings=HuggingFaceEmbeddings()

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_7960\592125342.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [10]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [11]:
query_result = embeddings.embed_query("How are you")
len(query_result)

384

In [ ]:
query_result

#**06: Convert the Text Chunks into Embeddings and Create a Knowledge Base**

In [ ]:
# PINECONE_API_KEY= '1db13105-964d-4965-9035-e09c00ad18a5'
# PINECONE_API_ENV= 'gcp-starter'

In [ ]:
# pinecone.init(
#    api_key=PINECONE_API_KEY,
#    environment=PINECONE_API_ENV
# )

In [ ]:
# index_name='llama'

In [ ]:
# vectorstore=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [ ]:
# vectorstore=Pinecone.from_documents(text_chunks, embeddings, index_name=index_name)

In [9]:
from langchain.vectorstores import FAISS
vectorstore = FAISS.from_documents(text_chunks, embeddings)

#**07: Create a Large Language Model (LLM) Wrapper**

In [ ]:
# notebook_login()

In [10]:
model = "meta-llama/Llama-2-7b-chat-hf"

In [11]:
# from transformers import BitsAndBytesConfig


# bnb_config = BitsAndBytesConfig(load_in_4bit=True)

# tokenizer = AutoTokenizer.from_pretrained(model, token=token)


# model = AutoModelForCausalLM.from_pretrained(model,
#                                              device_map='auto',
#                                              dtype=torch.float16,
#                                              token=token,
#                                               load_in_8bit=True,
#                                               # load_in_4bit=True,
#                                              )
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(load_in_4bit=True)

tokenizer = AutoTokenizer.from_pretrained(model, token=token)
model = AutoModelForCausalLM.from_pretrained(
    model,
    device_map="auto",
    quantization_config=bnb_config,
    token=token
)

c:\d\generative AI\LLaMA-Reader\venv\Lib\site-packages\transformers\models\auto\tokenization_auto.py:1010: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
c:\d\generative AI\LLaMA-Reader\venv\Lib\site-packages\transformers\models\auto\auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:23<00:00, 11.95s/it]


In [12]:
pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

Device set to use cuda:0


In [13]:
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_7960\596095089.py:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})


In [14]:
llm.predict("Please provide a concise summary of the Book Harry Potter")

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_7960\1177971522.py:1: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm.predict("Please provide a concise summary of the Book Harry Potter")


"Please provide a concise summary of the Book Harry Potter and the Philosopher's Stone by J. nobody knew that the stone was actually a magical object that had the power to grant eternal life to whoever possessed it.\nHarry Potter is an orphaned boy who discovers he is a wizard and begins attending Hogwarts School of Witchcraft and Wizardry. Rowling's debut novel, Harry Potter and the Philosopher's Stone, was first published in 1997.\nIn the story, Harry learns that he is the son of two powerful wizards who were killed by the dark wizard, Lord Voldemort. Harry is famous in the wizarding world for being the only person to survive a killing curse from Voldemort, and for being the chosen one to defeat the dark lord.\nHarry's life is forever changed when he discovers he is a wizard and begins attending Hogwarts School of Witchcraft and Wizardry. The novel introduces readers to a magical world filled with talking animals, wizards, and witches.\nThe story follows Harry as he navigates his fir

#**08: Initialize the Retrieval QA with Source Chain**

In [15]:
from langchain.chains import RetrievalQA

In [20]:
query = "How good is Vicuna?"

In [21]:
docs = vectorstore.similarity_search(query, k=3)

In [22]:
docs

[Document(id='5841ac45-3cba-4249-8fb6-d58e94908ce0', metadata={'source': 'https://lmsys.org/blog/2023-03-30-vicuna/'}, page_content='Vicuna (generated by stable diffusion 2.1)\nAccording to a fun and non-scientific evaluation with GPT-4. Further rigorous evaluation is needed.\nHow Good is Vicuna?\nAfter fine-tuning Vicuna with 70K user-shared ChatGPT conversations, we discover that Vicuna becomes capable of generating more detailed and well-structured answers compared to Alpaca (see examples below), with the quality on par with ChatGPT.'),
 Document(id='7680be23-3265-4827-82a6-a4238dbea6b4', metadata={'source': 'https://lmsys.org/blog/2023-03-30-vicuna/'}, page_content='LMSYS ORG\nProjectsBlogAboutDonationsChatbot Arena (graduated)\nVicuna: An Open-Source Chatbot Impressing GPT-4 with 90%* ChatGPT Quality\nby: The Vicuna Team, Mar 30, 2023\nWe introduce Vicuna-13B, an open-source chatbot trained by fine-tuning LLaMA on user-shared conversations collected from ShareGPT. Preliminary eval

In [16]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever())

In [17]:
query = "How good is Vicuna?"
qa.run(query)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_7960\2118127610.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa.run(query)


"Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nVicuna (generated by stable diffusion 2.1)\nAccording to a fun and non-scientific evaluation with GPT-4. Further rigorous evaluation is needed.\nHow Good is Vicuna?\nAfter fine-tuning Vicuna with 70K user-shared ChatGPT conversations, we discover that Vicuna becomes capable of generating more detailed and well-structured answers compared to Alpaca (see examples below), with the quality on par with ChatGPT.\n\nLMSYS ORG\nProjectsBlogAboutDonationsChatbot Arena (graduated)\nVicuna: An Open-Source Chatbot Impressing GPT-4 with 90%* ChatGPT Quality\nby: The Vicuna Team, Mar 30, 2023\nWe introduce Vicuna-13B, an open-source chatbot trained by fine-tuning LLaMA on user-shared conversations collected from ShareGPT. Preliminary evaluation using GPT-4 as a judge shows Vicuna-13B achieves more than 90%* quality of OpenAI ChatGPT an

In [25]:
query = "How does Llama 2 outperforms other models"
qa.run(query)

"Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nTable 1. Comparison between several notable models\nModel Name LLaMA Alpaca Vicuna Bard/ChatGPT Dataset Publicly available datasets (1T token) Self-instruct from davinci-003 API (52K samples) User-shared conversations (70K samples) N/A Training code N/A Available Available N/A Evaluation metrics Academic benchmark Author evaluation GPT-4 assessment Mixed Training cost (7B) 82K GPU-hours $500 (data) + $100 (training) $140 (training) N/A Training cost (13B) 135K GPU-hours N/A $300 (training) N/A\nTraining\nVicuna is created by fine-tuning a LLaMA base model using approximately 70K user-shared conversations gathered from ShareGPT.com with public APIs. To ensure data quality, we convert the HTML back to markdown and filter out some inappropriate or low-quality samples. Additionally, we divide lengthy conversations into smalle

In [26]:
query = "What is is stableLM?"
qa.run(query)

"Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nStability AI Launches the First of its Stable LM Suite of Language Models\nProduct\nApr 19\nWritten By Joshua Lopez\nToday, Stability AI released a new open source language model, Stable LM. The Alpha version of the model is available in 3 billion and 7 billion parameters, with 15 billion to 65 billion parameter models to follow. Developers can freely inspect, use, and adapt our Stable LM base models for commercial or research purposes, subject to the terms of the CC BY-SA-4.0 license.\nIn 2022, Stability AI drove the public release of Stable Diffusion, a revolutionary image model representing a transparent, open, and scalable alternative to proprietary AI. With the launch of the Stable LM suite of models, Stability AI is continuing to make foundational AI technology accessible to all. Our Stable LM models can generate te

In [ ]:
while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = qa({'query': user_input})
  print(f"Answer: {result['result']}")